<a href="https://colab.research.google.com/github/mabdulghofur142/Instagram-Grid-Cropper/blob/main/Code/Instagram_Grid_Cropper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instagram Grid Cropper

## Modul yang dibutuhkan

In [ ]:
from PIL import Image
import io
import zipfile
from fractions import Fraction
import math

## Code

In [ ]:
def place_on_frame(image, frame_size=(2400, 3000), position=0, round=0, frame=None):
    if round == 0 or frame is None:
        frame = Image.new("RGB", frame_size, (0, 0, 0))
    img_width, img_height = image.size

    if position == 1:
        x_offset = frame_size[0] - img_width
    elif position == -1:
        x_offset = 0
    else:  # center
        x_offset = (frame_size[0] - img_width) // 2

    y_offset = (frame_size[1] - img_height) // 2
    frame.paste(image, (x_offset, y_offset))
    return frame

def labelling(rows, length):
    my_map = {}
    for i in range(length):
        row_image = (i % rows) + 1
        col_image = (i // rows) +1
        my_map[i] = str(row_image)+"_"+str(col_image)
    return my_map

def process_image(input_path, coloms=3, rows=1, output_zip_path='cropped_images.zip', format_upload_feed = "4/5", format_preview = "3/4"):
    img = Image.open(input_path)
    width, height = img.size

    pecahan = Fraction(width, height)
    pecahan2 = Fraction(coloms*3, rows*4)
    print(f"Is the ratio already in accordance with the Instagram preview format? {pecahan == pecahan2}")

    gap_image = Fraction(format_upload_feed)-Fraction(format_preview)

    crop_width, crop_height = width // coloms, height // rows

    cropped_images = []

    if coloms == 2:
        cols = [0,2]
    elif coloms == 1:
        cols = [0]
    elif coloms == 3:
        cols = range(coloms)

    for col in cols:
        if col == 0:
            left = col * crop_width
            right = left + crop_width + math.ceil((gap_image/rows/2) * height)
            position = 1
            if coloms == 1:
                position = 0
        elif col == 1:
            left1 = col * crop_width
            left = left1 - int((gap_image/rows/2) * height)
            right = left1 + crop_width + math.ceil((gap_image/rows/2) * height)
            position = 0
        elif col == 2:
            left1 = col * crop_width
            left = left1 - int((gap_image/rows/2) * height)
            right = left1 + crop_width
            position = -1

        left, right = max(0, left), min(width, right)
        upper, lower = 0, height
        frame_size = (crop_width + (int((gap_image/rows) * height)), height)
        cropped = img.crop((left, upper, right, lower))
        framed_cropped = place_on_frame(cropped, frame_size=frame_size, position=position)
        if col == 0 or col == 2 or cols == [0]:
            if cols == [0]:
                col = [0,2]
                position = 1
            else:
                col = [col]

            flipped = cropped.transpose(Image.FLIP_LEFT_RIGHT)
            flipped_width, flipped_height = flipped.size

            for i in col:
                position=-position
                if i == 0:
                    left = flipped_width - math.ceil((gap_image/rows/2) * flipped_height)
                    right = flipped_width
                else:
                    left = 0
                    right = math.ceil((gap_image/rows/2) * flipped_height)

                left, right = max(0, left), min(width, right)

                flipped_cropped = flipped.crop((left, 0, right, flipped_height))
                framed_cropped = place_on_frame(flipped_cropped, frame_size=frame_size, position=position, round=1, frame=framed_cropped)
        # display(framed_cropped)
        if rows > 1:
            w, h = framed_cropped.size
            list_image = []
            for row in range(rows):
                framed_cropped_new = framed_cropped.crop((0, row * crop_height, w, (row + 1) * crop_height))
                list_image.append(framed_cropped_new)
            cropped_images = cropped_images + list_image

        if rows == 1:
            cropped_images.append(framed_cropped)

    my_label = labelling(rows, len(cropped_images))
    # Simpan ke file ZIP
    with zipfile.ZipFile(output_zip_path, 'w', zipfile.ZIP_DEFLATED) as zip_file:
        for i, img in enumerate(cropped_images):
            img_bytes = io.BytesIO()
            img.save(img_bytes, format='JPEG')
            zip_file.writestr(f'cropped_{my_label[i]}.jpg', img_bytes.getvalue())

    print(f"ZIP file has been successfully created: {output_zip_path}")


# Testing

Testing using Data Image from my github: https://github.com/mabdulghofur142/Instagram-Grid-Cropper/tree/main/Image

## Test 1x3


In [ ]:
if __name__ == '__main__':
    input_image_path = '/content/example 1x3 (3375 x 1500 piksel).png' # image path
    output_zip_name = 'cropped example 1x3.zip' # zip name
    coloms = 3
    rows = 1
    process_image(
        input_path=input_image_path,
        coloms=coloms,
        rows=rows,
        output_zip_path=output_zip_name,
        format_upload_feed="4/5",     # Target rasio carousel Instagram
        format_preview="3/4"    # Rasio asli gambar
    )


Is the ratio already in accordance with the Instagram preview format? True
ZIP file has been successfully created: cropped example 1x3.zip


In [ ]:
from google.colab import files
files.download(output_zip_name)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Test 2x3


In [ ]:
if __name__ == '__main__':
    input_image_path = '/content/example 2x3 (3375x3000).png'
    output_zip_name_2 = 'cropped example 2x3.zip'
    coloms = 3
    rows = 2
    process_image(
        input_path=input_image_path,
        coloms=coloms,
        rows=rows,
        output_zip_path=output_zip_name_2,
        format_upload_feed="4/5",     # Target rasio carousel Instagram
        format_preview="3/4"    # Rasio asli gambar
    )


Is the ratio already in accordance with the Instagram preview format? True
ZIP file has been successfully created: cropped example 2x3.zip


In [ ]:
from google.colab import files
files.download(output_zip_name_2)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Test 3x3


In [ ]:
if __name__ == '__main__':
    input_image_path = '/content/example 3x3 (3375 x 4500 piksel).png'
    output_zip_name_3 = 'cropped example 3x3.zip'
    coloms = 3
    rows = 3
    process_image(
        input_path=input_image_path,
        coloms=coloms,
        rows=rows,
        output_zip_path=output_zip_name_3,
        format_upload_feed="4/5",     # Target rasio carousel Instagram
        format_preview="3/4"    # Rasio asli gambar
    )


Is the ratio already in accordance with the Instagram preview format? True
ZIP file has been successfully created: cropped example 3x3.zip


In [ ]:
from google.colab import files
files.download(output_zip_name_3)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>